In [2]:
import pandas as pd
import numpy as np
import folium
import plotly.express as px
import seaborn as sns
from folium import plugins
from folium.plugins import HeatMap

# Load processed dataframe

In [3]:
df_boston = pd.read_csv('./data/prepared/rides_data_prepared.csv', dtype={'start_station_id': np.int64, 'end_station_id': 'string', 'end_station_name': 'string', 'start_station_name': 'string', 'bike_id': np.int64, 'user_type': 'string'})
df_boston['start_time'] = pd.to_datetime(df_boston['start_time'], format='%Y-%m-%d %X')
df_boston['end_time'] = pd.to_datetime(df_boston['end_time'], format='%Y-%m-%d %X')
df_boston.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1106095 entries, 0 to 1106094
Data columns (total 14 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   start_time               1106095 non-null  datetime64[ns]
 1   end_time                 1106095 non-null  datetime64[ns]
 2   start_station_id         1106095 non-null  int64         
 3   end_station_id           1106095 non-null  string        
 4   start_station_name       1106095 non-null  string        
 5   end_station_name         1106095 non-null  string        
 6   bike_id                  1106095 non-null  int64         
 7   user_type                1106095 non-null  string        
 8   trip_length              1106095 non-null  float64       
 9   start_station_latitude   1106095 non-null  float64       
 10  start_station_longitude  1106095 non-null  float64       
 11  end_station_latitude     1106095 non-null  float64       
 12  

In [13]:
df_boston.head()

,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type,trip_length,start_station_latitude,start_station_longitude,end_station_latitude,end_station_longitude,date_time
0,2015-01-01 00:21:44,2015-01-01 00:30:47,115,96,Porter Square Station,Cambridge Main Library at Broadway / Trowbridg...,277,Subscriber,543.0,42.387995,-71.119084,42.373379,-71.111075,2015-01-01 00:00:00
1,2015-01-01 00:27:03,2015-01-01 00:34:21,80,95,MIT Stata Center at Vassar St / Main St,Cambridge St - at Columbia St / Webster Ave,648,Subscriber,438.0,42.361962,-71.092053,42.372969,-71.094445,2015-01-01 00:00:00
2,2015-01-01 00:31:31,2015-01-01 00:35:46,91,68,One Kendall Square at Hampshire St / Portland St,Central Square at Mass Ave / Essex St,555,Subscriber,255.0,42.366277,-71.091690,42.365070,-71.103100,2015-01-01 01:00:00
3,2015-01-01 00:53:46,2015-01-01 01:00:58,115,96,Porter Square Station,Cambridge Main Library at Broadway / Trowbridg...,1307,Subscriber,432.0,42.387995,-71.119084,42.373379,-71.111075,2015-01-01 01:00:00
4,2015-01-01 01:07:06,2015-01-01 01:19:21,105,88,Lower Cambridgeport at Magazine St/Riverside Rd,Inman Square at Vellucci Plaza / Hampshire St,177,Customer,735.0,42.356954,-71.113687,42.374035,-71.101427,2015-01-01 01:00:00


# Station analysis

## Starts stations volume  per year
Visualization which contains information on how many rides start at a respective station per year

In [20]:
#prepare dataframe that only includes start stations used by subscribers and customers
df_start = df_boston.groupby(['start_station_id','start_station_latitude', 'start_station_longitude', 'user_type']).size().reset_index(name='counts_start')
df_start_s = df_start[df_start['user_type'] == 'Subscriber']
df_start_c = df_start[df_start['user_type'] == 'Customer']

In [29]:
#visualize station use of start stations by subscribers
color_scale = [(0, 'orange'), (1,'red')]

fig = px.scatter_mapbox(df_start_s, lat="start_station_latitude", lon="start_station_longitude", color='counts_start',
                        color_continuous_scale=color_scale,size='counts_start', zoom=12, height=300, width=800)


fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [35]:
#visualize station use of start stations by customers
color_scale = [(0, 'orange'), (1,'red')]

fig = px.scatter_mapbox(df_start_c, lat="start_station_latitude", lon="start_station_longitude", color='counts_start',
                        color_continuous_scale=color_scale,size='counts_start', zoom=12, height=300, width=800)


fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

We can make out three major hotspots for subscribers: The city center around down town, stations around the MIT, and stations around the harvard university. Where the most used stations are near the universitys.
In contrast the regular customers tend to use the bikes mostly in the downtown area near the train stations and the city center. 


## End station rides volume
Visualization which contains information on how many rides end at a respective station per year

In [36]:
#prepare dataframe that only includes end stations used by subscribers and customers
df_end = df_boston.groupby(['end_station_id','end_station_latitude', 'end_station_longitude', 'user_type']).size().reset_index(name='counts_end')
df_end_s = df_end[df_end['user_type'] == 'Subscriber']
df_end_c = df_end[df_end['user_type'] == 'Customer']

In [38]:
#visualize station use of start stations by subscriber
color_scale = [(0, 'orange'), (1,'red')]

fig = px.scatter_mapbox(df_end_s, lat="end_station_latitude", lon="end_station_longitude", color='counts_end',
                        color_continuous_scale=color_scale,size='counts_end', zoom=12, height=300, width=800)


fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [39]:
#visualize station use of start stations by customers
color_scale = [(0, 'orange'), (1,'red')]

fig = px.scatter_mapbox(df_end_c, lat="end_station_latitude", lon="end_station_longitude", color='counts_end',
                        color_continuous_scale=color_scale,size='counts_end', zoom=12, height=300, width=800)


fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

There seems to be no big difference between the amount of trips ended and started in certain areas which might indicate that people that decide to rent a bike also choose it again later when leaving their destination. 

Also the observations of customer and subscriber use for start stations does not change for end stations.

## Volume share per station 
What is the percentage of total of total rides a station is used? 

In [7]:
#share of different stations, most frequently used stations
df_station_share = df_start.copy()
df_station_share['total_rides'] = df_start['counts_start'] + df_end['counts_end']
yearly_total_rides = sum(df_station_share['total_rides'])
df_station_share['share'] = (df_station_share['total_rides'] / yearly_total_rides)*100
df_station_share.sort_values('share', ascending=False).head(5)

,start_station_id,start_station_latitude,start_station_longitude,counts_start,total_rides,share
75,81,42.352409,-71.062679,10155,41190,1.861956
19,22,42.352175,-71.055547,30543,37438,1.692350
55,60,42.360835,-71.070840,16915,32665,1.476591
121,130,42.317225,-71.065421,742,30400,1.374204
61,67,42.358100,-71.093198,29335,29912,1.352144


## Station Distribution in Boston

In [43]:
def heat_map(df, station_id):
    # define new map
    heat_map = folium.Map(
        location=[df['start_station_latitude'].mean(), df['start_station_longitude'].mean()], 
        tiles='OpenStreetMap', 
        zoom_start=13, 
        control_scale=True, 
        max_zoom=20,
        )
    # add heat map
    heat_data = [[row['start_station_latitude'],row['start_station_longitude']] for index, row in df.iterrows()]
    HeatMap(heat_data).add_to(heat_map)
    #heat map for destination points looks pretty much identical
    return heat_map
m = heat_map(df_start, np.array(df_start['start_station_id']))

m

# Most popular routes  
This section tries to explain which routes are most often used.

In [9]:
df_boston.head()

,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type,trip_length,start_station_latitude,start_station_longitude,end_station_latitude,end_station_longitude,date_time
0,2015-01-01 00:21:44,2015-01-01 00:30:47,115,96,Porter Square Station,Cambridge Main Library at Broadway / Trowbridg...,277,Subscriber,543.0,42.387995,-71.119084,42.373379,-71.111075,2015-01-01 00:00:00
1,2015-01-01 00:27:03,2015-01-01 00:34:21,80,95,MIT Stata Center at Vassar St / Main St,Cambridge St - at Columbia St / Webster Ave,648,Subscriber,438.0,42.361962,-71.092053,42.372969,-71.094445,2015-01-01 00:00:00
2,2015-01-01 00:31:31,2015-01-01 00:35:46,91,68,One Kendall Square at Hampshire St / Portland St,Central Square at Mass Ave / Essex St,555,Subscriber,255.0,42.366277,-71.091690,42.365070,-71.103100,2015-01-01 01:00:00
3,2015-01-01 00:53:46,2015-01-01 01:00:58,115,96,Porter Square Station,Cambridge Main Library at Broadway / Trowbridg...,1307,Subscriber,432.0,42.387995,-71.119084,42.373379,-71.111075,2015-01-01 01:00:00
4,2015-01-01 01:07:06,2015-01-01 01:19:21,105,88,Lower Cambridgeport at Magazine St/Riverside Rd,Inman Square at Vellucci Plaza / Hampshire St,177,Customer,735.0,42.356954,-71.113687,42.374035,-71.101427,2015-01-01 01:00:00


In [10]:
#number of trips between a start and end station

df_trips = df_boston.groupby(['start_station_id','end_station_id', 'start_station_latitude', 'start_station_longitude', 'end_station_latitude', 'end_station_longitude']).size().reset_index(name='count_trips')

df_trips = df_trips.sort_values('count_trips', ascending = False)

df_trips.index = range(0, len(df_trips))

df_trips.head(10)

,start_station_id,end_station_id,start_station_latitude,start_station_longitude,end_station_latitude,end_station_longitude,count_trips
0,118,100,42.397828,-71.130516,42.396969,-71.123024,3011
1,100,118,42.396969,-71.123024,42.397828,-71.130516,2740
2,114,100,42.402763,-71.126908,42.396969,-71.123024,2042
3,67,53,42.358100,-71.093198,42.350851,-71.089886,2022
4,40,22,42.363871,-71.050877,42.352175,-71.055547,1979
5,67,68,42.358100,-71.093198,42.365070,-71.103100,1836
6,53,67,42.350851,-71.089886,42.358100,-71.093198,1783
7,22,40,42.352175,-71.055547,42.363871,-71.050877,1740
8,68,67,42.365070,-71.103100,42.358100,-71.093198,1607
9,20,22,42.359770,-71.051601,42.352175,-71.055547,1521


In [11]:
df_top_trips = df_trips.head(47).copy() #most common trips ignoring direction of trip, filter is 47 as we only use values above 1000 

for j in range(0, len(df_top_trips)):
    if j < len(df_top_trips):
        for i in range (0, len(df_top_trips)):
            if i < len(df_top_trips):
                if df_top_trips.at[j,'start_station_id'] == int(df_top_trips.at[i, 'end_station_id']) and df_top_trips.at[i,'start_station_id'] == int(df_top_trips.at[j, 'end_station_id']):
                    df_top_trips.at[j, 'count_trips'] = df_top_trips.at[i, 'count_trips'] + df_top_trips.at[j, 'count_trips']
                    df_top_trips = df_top_trips.drop(i)
                    df_top_trips.index = range(0, len(df_top_trips))


df_top_trips = df_top_trips.sort_values('count_trips', ascending = False)
df_top_trips.index = range(0, len(df_top_trips))
df_top_trips = df_top_trips.rename(columns = {'start_station_id' : 'Station_A', 'end_station_id' : 'Station_B'})
df_top_trips = df_top_trips.head(16) #big gap between 16 and 17 from over 2100 to 1300
df_top_trips


,Station_A,Station_B,start_station_latitude,start_station_longitude,end_station_latitude,end_station_longitude,count_trips
0,118,100,42.397828,-71.130516,42.396969,-71.123024,5751
1,67,53,42.358100,-71.093198,42.350851,-71.089886,3805
2,40,22,42.363871,-71.050877,42.352175,-71.055547,3719
3,67,68,42.358100,-71.093198,42.365070,-71.103100,3443
4,114,100,42.402763,-71.126908,42.396969,-71.123024,3191
5,20,22,42.359770,-71.051601,42.352175,-71.055547,2980
6,67,75,42.358100,-71.093198,42.363465,-71.100573,2886
7,67,74,42.358100,-71.093198,42.373268,-71.118579,2692
8,67,107,42.358100,-71.093198,42.362500,-71.088220,2603
9,22,109,42.352175,-71.055547,42.365942,-71.060515,2380


In [12]:
#mapping the top trips 


avg_lat = df_trips['start_station_latitude'].mean()
avg_long = df_trips['start_station_longitude'].mean()

m = folium.Map(location=[avg_lat, avg_long], zoom_start=15)

points = []


for i in range(0,len(df_top_trips)):
    points.append([df_top_trips.iloc[i]['start_station_latitude'],df_top_trips.iloc[i]['start_station_longitude']])
    points.append([df_top_trips.iloc[i]['end_station_latitude'],df_top_trips.iloc[i]['end_station_longitude']])
    folium.PolyLine(points, color="red", weight=2.5, opacity=1).add_to(m)
    folium.Marker([df_top_trips.iloc[i]['start_station_latitude'],df_top_trips.iloc[i]['start_station_longitude']],popup=(df_top_trips.iloc[i]['Station_A'], df_top_trips.iloc[i]['count_trips' ]), icon = folium.Icon(color='blue',prefix='fa', icon='bicycle')).add_to(m)
    folium.Marker([df_top_trips.iloc[i]['end_station_latitude'],df_top_trips.iloc[i]['end_station_longitude']],popup=(df_top_trips.iloc[i]['Station_B'], df_top_trips.iloc[i]['count_trips' ]), icon = folium.Icon(color='blue',prefix='fa', icon='bicycle')).add_to(m)
    points = [] #if points is not reseted then each station will be connected with every other station in array

m 

#findings: near universities (harvard and mit) as well as around citycenter(haymarket) and the trainstations (north and south)